# West Coastal Plain & Ouachitas Forested Wetland Birds

This is an indicator for the 2023 base blueprint.

The WGCPO Forested Wetland Decision Support Model prioritizes existing forested wetland habitat according to habitat suitability for the five Forested Wetland Umbrella Species (Acadian flycatcher, Kentucky warbler, prothonotary warbler, red-shouldered hawk, and yellow-throated warbler). 

Created by Amy Keister, last run by Amy Keister on 1 June 2023. I took 3 minutes to run this code.

In [1]:
import os
import arcpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\WestCoastalPlainAndOuachitasFor\WestCo.gdb"

In [4]:
# define final indicator outputs
Out = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\WestCoastalPlainAndOuachitasFor\WestCoastalPlainAndOuachitasForestedWetlandBirds.tif"

In [5]:
# define sub-indicator outputs

In [6]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [7]:
# define inputs
BLH= r"F:\GIS_DATA\DecisionSupportTools\LMVJV\Final_combinedHSI_WGCPO_BLH_utm83\NEW_Final_combinedHSI_WGCPO_BLH_utm83_clpd2_0917.img"

## Start Analysis

In [8]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [9]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\WestCoastalPlainAndOuachitasFor\WestCo.gdb


## bring in LMVJV Bottomland Hardwood Bird priority data for the WGCPO

In [10]:
# Bring in the LMVJV tif
arcpy.conversion.RasterToGeodatabase(BLH, OutWorkspace, '')

<Result ''>

In [11]:
# reclassify to bin

# ArcGIS reclass includes the second value in the class for example,
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)

with arcpy.EnvManager(outputCoordinateSystem=sr, cellSize=Rextent):
    out_raster = arcpy.sa.Reclassify("NEW_Final_combinedHSI_WGCPO_BLH_utm83_clpd2_0917", "VALUE", "0 0 0;0.00001 0.2 1;0.2 0.4 2;0.4 0.6 3;0.6 0.8 4;0.8 1 5", "DATA"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [10]:
# clip to SE 2022 Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("SEMask2")

In [11]:
# export as .tif 
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("SEMask2", Out, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [12]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 5:
        return '5 = High habitat suitability for forested wetland bird umbrella species (Acadian flycatcher, Kentucky warbler, yellow-throated warbler, prothonotary warbler, red-shouldered hawk) (score >80)'
    elif value == 4:
        return '4 = Medium-high habitat suitability (score >60-80)'
    elif value == 3:
        return '3 = Medium habitat suitability (score >40-60)'
    elif value == 2:
        return '2 = Medium-low habitat suitability (score >20-40)'
    elif value == 1:
        return '1 = Low habitat suitability for forested wetland bird umbrella species (score >0-20)'
    elif value == 0:
        return '0 = Not suitable for forested wetland bird umbrella species (score =0)'
"""

In [13]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(Out, "descript", "Reclass(!Value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\WestCoastalPlainAndOuachitasFor\\WestCoastalPlainAndOuachitasForestedWetlandBirds.tif'>

In [14]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 5:
		return 22
	if Value == 4:
		return 124
	if Value == 3:
		return 197
	if Value == 2:
		return 232
	if Value == 1:
		return 234
	else:
		return 255
		
def Reclass2(Value):
	if Value == 5:
		return 74
	if Value == 4:
		return 117
	if Value == 3:
		return 146
	if Value == 2:
		return 202
	if Value == 1:
		return 224
	else:
		return 255
		
def Reclass3(Value):
	if Value == 5:
		return 137
	if Value == 4:
		return 153
	if Value == 3:
		return 156
	if Value == 2:
		return 170
	if Value == 1:
		return 175
	else:
		return 255
		
"""

In [15]:
# calculate Red field
arcpy.management.CalculateField(Out, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(Out, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(Out, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\WestCoastalPlainAndOuachitasFor\\WestCoastalPlainAndOuachitasForestedWetlandBirds.tif'>

In [18]:
end = time.time()
print(end - start)

209.58286881446838
